In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import spinmob as sm
import mcphysics as mphys
from scipy.optimize import curve_fit

In [15]:
#Adds together 1 hour of calibration data for a given source
def Sum(source):
    yData = [0]*2048
    i=1
    j=0
    while i <= 12:
        while j < 10:
            yData += mphys.data.load_chn('Calibration/' + source + '/' + str(i) + '/' + source + '/' + source + 
                                         ' 00' + str(j) + '.Chn')[1]
            j += 1
        i += 1
    return yData

#Gaussian, where m=peak, o=variance, A=amplitude, B=Linear Coeff., C=Linear Offset
def Gaussian(x, m, o, A, B, C):
    return A*np.exp(-0.5*(((x-m)/o)**2)) + B*x + C

#Function for linear curve-fit of peaks for each channel (i.e. for final callibration curve)
def Linear(x, a, b):
    return a*x+b

#Curve fits a Gaussian to a given emitter's callibration data curve. The interval determines the width of data
#which the Gaussian fits around. 
def Fit(input_databox, interval, guess):
    xData = np.arange(0, 2048)
    param, cov = curve_fit(Gaussian, xData[guess-interval:guess+interval], 
                           input_databox[guess-interval:guess+interval], p0=[guess, 10, 10, 10, 10])
    error = np.sqrt(np.diag(cov))
#    print(param), print(error)
    return [param, error]

def Chi(ydata, yFit):
    return ((ydata-yFit)**2)/(ydata**2)

Cs = Fit(Sum('Cs-137'), 300, 1742)
Ba = Fit(Sum('Ba-133'), 300, 850)
Co = Fit(Sum('Co-57'), 300, 375)
Na = Fit(Sum('Na-22'), 300, 1360)

yvalues = [662.1, 80, 122, 511.0]
xvalues = [Cs[0][0], Ba[0][0], Co[0][0], Na[0][0]]
errvalues = [Cs[1][0], Ba[1][0], Co[1][0], Na[1][0]]

linearFit_par, linearFit_cov = curve_fit(Linear, xvalues, yvalues, sigma = errvalues)

plt.plot(np.arange(0, 2048), Linear(np.arange(0, 2048), *linearFit_par))
plt.scatter(xvalues, yvalues)
print(linearFit_par)

[  0.3825181  -14.84090682]
